# Custom Extensions for Leifer Lab Datasets

In [1]:
export_ext = True

In [2]:
import pynwb
from pynwb.spec import NWBNamespaceBuilder, NWBLinkSpec,NWBGroupSpec, NWBDatasetSpec, NWBAttributeSpec
        
        

## Build neural data NWB extensions

In [3]:
ns_path = "leiferlab.namespace.yaml"
ext_source = "leiferlab.extensions.yaml"



In [4]:
# This builds the name space
ns_builder = NWBNamespaceBuilder(doc="Extensions to be used for Scholz datasets from Leifer lab",
                                name="leiferlab", # reference to above 
                                author='Vergil R. Haynes',
                                version=pynwb.__version__)



In [5]:
types = ['NWBDataInterface','TimeSeries','SpatialSeries']

for this_type in types:
    ns_builder.include_type(data_type=this_type,
                  namespace='core')
    
    

#### NeuronPositionTable to replace PlaneSegmentation
<u>Notes</u>:
> These datasets do not contain image stacks to be used to build ROI DynamicTableRegion. Alternatively, we build a neuronal position table which can link all associated data elements. 
>
> See use of ImagingPlane, PlaneSegmentation and ImageSegmentation here (https://pynwb.readthedocs.io/en/stable/_modules/pynwb/ophys.html)

In [6]:
# What new attributes to include associated with the table
attributes = [NWBAttributeSpec(name='origin_coords',
                              doc='Physical location of the first element of the neural volume such as (0,0,0) for 3D data',
                              dtype='float64',
                              shape=(3,),
                              required=False),
             NWBAttributeSpec(name='description',
                             doc='Description of this table',
                             dtype='text',
                             required=True)]

# Small datasets for table (column oriented)
datasets = [NWBDatasetSpec(name='neuron_id',
                              doc='Neuron IDs under specified ordering',
                              dtype='numeric',
                              dims=['num_neurons',],
                             shape=(None,)),

            NWBDatasetSpec(name='x_pos',
                               doc='Neuron 1D X-position previously extracted from image stacks',
                               dtype='numeric',
                               dims=['num_neurons'],
                               shape=(None,)),
                           
            NWBDatasetSpec(name='y_pos',
                               doc='Neuron 1D Y-position previously extracted from image stacks',
                               dtype='numeric',
                               dims=['num_neurons'],
                               shape=(None,)),
                           
            NWBDatasetSpec(name='z_pos',
                               doc='Neuron 1D Z-position previously extracted from image stacks',
                               dtype='numeric',
                               dims=['num_neurons'],
                               shape=(None,))]

# this the actual extension
ext = NWBGroupSpec('A custom NeuronPositionTable for Leifer lab',
                  datasets=datasets,
                  attributes=attributes,
                  neurodata_type_inc='NWBDataInterface',
                  neurodata_type_def='NeuronPositionTable')

# add it to the name space builder
ns_builder.add_spec(ext_source,ext)



#### ChannelGroup to contain both optical channels

In [7]:
attributes = [NWBAttributeSpec(name='description',
                             doc='Description of this group pertaining to differences in channels',
                             dtype='text',
                             required=True)]


links = [NWBLinkSpec(name='device',
                          doc='Link to Device object used to record',
                          target_type='Device'),
         NWBLinkSpec(name='green_optical_channel',
                          doc='Link to specific optical channel for GCAMP6',
                          target_type='OpticalChannel'),
         NWBLinkSpec(name='red_optical_channel',
                          doc='Link to specific optical channel for red',
                          target_type='OpticalChannel')]



ext = NWBGroupSpec('A custom ChannelGroup for Leifer lab',
                 attributes=attributes,
                  links=links,
                  neurodata_type_inc='NWBDataInterface',
                  neurodata_type_def='ChannelGroup')

ns_builder.add_spec(ext_source,ext)



#### ResponseSeries to replace ROIResponseSeries


In [8]:
attributes = [NWBAttributeSpec(name='neuron_id',
                             doc='Neuron IDs under specified ordering',
                             dtype='int',
                             required=True),
             NWBAttributeSpec(name='description',
                             doc='Description of time series',
                             dtype='text',
                             default_value='no description',
                             required=False),
             NWBAttributeSpec(name='comments',
                             doc='Human-readable comments about the timeseries',
                             dtype='text',
                             default_value='no comment',
                             required=False),]
#              NWBAttributeSpec(name='optical_channel',
#                              doc='Description of optical channel used',
#                              dtype='text',
#                              default_value='no optical_channel',
#                              required=False)]


datasets = [NWBDatasetSpec(name='data',
                          doc='Neural response signal',
                           dtype='numeric',
                           dims=[['num_times'],['num_times','num_neurons']],
                          shape=[None,[None,None]]),
            NWBDatasetSpec(name='timestamps',
                          doc='Timestamps for samples stored in data, in seconds',
                          dtype='float64',
                          dims=['num_times'],
                          shape=[None])]


links = [NWBLinkSpec(name='optical_channel',
                          doc='Link to specific channels used for this timeseries',
                          target_type='OpticalChannel',
                          quantity='*')] # doesn't have to have it



# this the actual extension
ext = NWBGroupSpec('A custom ResponseSeries for Leifer lab',
                   datasets=datasets,
                  attributes=attributes,
                   links=links,
                  neurodata_type_inc='TimeSeries',
                  neurodata_type_def='ResponseSeries')

ns_builder.add_spec(ext_source,ext)



## Build behavioral data NWB extensions

In [9]:
attributes = [NWBAttributeSpec(name='origin_coords',
                              doc='Physical location of the first element of imaging volume such as (0,0) for 2D data',
                              dtype='float64',
                              shape=(2,),
                              required=False),
              NWBAttributeSpec(name='coord_id',
                             doc='Coordinate ID used to specify a point along each centerline',
                             dtype='int',
                             required=True),
             NWBAttributeSpec(name='description',
                             doc='Description of time series',
                             dtype='text',
                             default_value='no description',
                             required=False),
             NWBAttributeSpec(name='comments',
                             doc='Human-readable comments about the timeseries',
                             dtype='text',
                             default_value='no comment',
                             required=False),]



datasets = [NWBDatasetSpec(name='data',
                          doc='Positional data corresponding to specific relative coordinate',
                           dtype='numeric',
                           dims=['num_times',['num_times','num_coords']],
                          shape=[None,[None,None]]),
            NWBDatasetSpec(name='timestamps',
                          doc='Timestamps for samples stored in data, in seconds',
                          dtype='float64',
                          dims=['num_times'],
                          shape=[None])]



# this the actual extension
ext = NWBGroupSpec('A custom CenterlineSeries for Leifer lab',
                   datasets=datasets,
                  attributes=attributes,
                  neurodata_type_inc='SpatialSeries',
                  neurodata_type_def='CenterlineSeries')

ns_builder.add_spec(ext_source,ext)



## Export custom extensions

In [10]:
if export_ext:
    ns_builder.export(ns_path)